In [ ]:
import openai
from transformers import AutoModelForCausalLM, AutoTokenizer


def generate_response(query, top_documents):
    #context = " ".join([doc['cleaned_summary'] for _, doc in top_documents])
    prompt = f"""
        Transform this question into a focused search query for academic papers.
        Remove question words, make it concise, and focus on key technical terms.

        Question: {query}

        Generate three different search queries, ordered by specificity (most specific first).
        Format: <query>search query here</query>
        """

    completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a research assistant."},
        {
            "role": "user",
            "content": prompt
        }
    ]
)
    return completion.choices[0].message.content

def generate_response_open_source(query, model_name):
    prompt = f"""
        Transform this question into a focused search query for academic papers.
        Remove question words, make it concise, and focus on key technical terms.

        Question: {query}

        Generate three different search queries, ordered by specificity (most specific first).
        Format: <query>search query here</query>
        """

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    messages=[
        {"role": "system", "content": "You are an SEO Expert."},
        {
            "role": "user",
            "content": prompt
        }
    ]

    text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
  )

    generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
  ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response


# Generate a response
response = generate_response_open_source("quantum computing", "Qwen/Qwen2.5-1.5B-Instruct")
print("\nGenerated Response:")
print(response)


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]


Generated Response:
1. "quantum computing algorithms"
  2. "quantum computers research"
  3. "quantum computing applications"


In [ ]:
def generate_field(title, model_name):
    prompt = f"""
        Predict the field to which this paper belongs to based on its title.

        Title: {title}

        Generate three different fields, ordered by specificity (most specific first).
        Format: [field1, field2, ...]
        """

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    messages=[
        {"role": "system", "content": "You are a research analyst."},
        {
            "role": "user",
            "content": prompt
        }
    ]

    text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
    )

    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
  )

    generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
  ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return response

response = generate_field("Classical and quantum reservoir computing: development and applications in machine learning", "Qwen/Qwen2.5-1.5B-Instruct")
print("\nGenerated Response:")
print(response)


Generated Response:
[Machine Learning, Quantum Computing, Reservoir Computing]


In [ ]:
from sentence_transformers import SentenceTransformer
from adapters import AutoAdapterModel
from transformers import AutoModelForCausalLM, AutoTokenizer

def encode_query(query):
  tokenizer = AutoTokenizer.from_pretrained('allenai/specter2_base')
  model = AutoAdapterModel.from_pretrained('allenai/specter2_base')

  model.load_adapter("allenai/specter2", source="hf", load_as="specter2", set_active=True)

  inputs = tokenizer(query, padding=True, truncation=True,
                                   return_tensors="pt", return_token_type_ids=False, max_length=512)
  output = model(**inputs)

  embeddings = output.last_hidden_state[:, 0, :]

  return embeddings

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
import requests
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import json

def similarity_search(query_embedding, paper_embedding):
  return float(cosine_similarity(query_embedding.detach().numpy(), np.vstack(paper_embedding).reshape(1,-1))[0][0])

def get_paper_title(query, num_papers = 1):
  base_url = "https://api.semanticscholar.org/graph/v1/paper/search"

  params = {
      "query": str(query),
      "fields": "title,fieldsOfStudy,embedding.specter_v2,abstract",
      "limit": num_papers
  }

  response = requests.get(base_url, params=params)

  if response.status_code == 200:
      data = response.json()
  else:
      print(f"Error: {response.status_code}")

  results = [{"title": data["data"][paper_num]['title'],
             "fields": data["data"][paper_num].get('fieldsOfStudy', []),
             "embedding": data["data"][paper_num]['embedding']['vector'],
             "abstract": data["data"][paper_num]['abstract']
             } for paper_num in range(num_papers)]

  return results


# query = "Convolutional neural networks for genomic sequence analysis"
# query_embedding = encode_query(query)

# num_papers = 10

# papers = get_paper_title(query, num_papers)

# similarity_scores = [similarity_search(query_embedding, paper["embedding"]) for paper in papers]

# papers_with_similarity = [{"title": paper["title"], "fields": paper["fields"], "citations": paper["citations"], "similarity": similarity} for paper, similarity in zip(papers, similarity_scores)]

# print(json.dumps(papers_with_similarity, indent=4))

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer
from typing import List, Dict, Optional, Tuple

class FieldEncoder(nn.Module):
    """
    Encodes field-of-study information using a transformer encoder layer
    to capture inter-field relationships.
    """
    def __init__(self, field_embedding_dim: int, nhead: int = 4, dim_feedforward: int = 512):
        super().__init__()
        self.transformer_encoder = nn.TransformerEncoderLayer(
            d_model=field_embedding_dim,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            batch_first=True
        )

        # Field embedding layer - we'll initialize this with pretrained embeddings later
        self.field_embeddings = nn.Embedding(num_embeddings=0, embedding_dim=field_embedding_dim)

    def forward(self, field_ids: torch.Tensor, attention_mask: Optional[torch.Tensor] = None) -> torch.Tensor:
        # [batch_size, max_fields, embedding_dim]
        field_embeds = self.field_embeddings(field_ids)

        if attention_mask is not None:
            # Convert boolean mask to float mask for transformer
            attention_mask = attention_mask.float().masked_fill(
                attention_mask == 0, float('-inf')
            ).masked_fill(attention_mask == 1, float(0.0))

        # Process through transformer encoder
        field_representations = self.transformer_encoder(field_embeds)
        return field_representations

In [ ]:
class CrossDisciplinaryRanker(nn.Module):
    """
    Bi-encoder model with cross-attention between paper content and fields of study.
    """
    def __init__(
        self,
        base_model_name: str = "allenai/specter2_base",
        field_embedding_dim: int = 768,
        projection_dim: int = 256
    ):
        super().__init__()

        # Document encoder (SPECTER2)
        self.document_encoder = AutoModel.from_pretrained(base_model_name)

        # Field encoder
        self.field_encoder = FieldEncoder(field_embedding_dim)

        # Cross-attention layer
        self.cross_attention = nn.MultiheadAttention(
            embed_dim=self.document_encoder.config.hidden_size,
            num_heads=8,
            batch_first=True
        )

        # Final projection layers
        self.content_projection = nn.Linear(self.document_encoder.config.hidden_size, projection_dim)
        self.field_projection = nn.Linear(field_embedding_dim, projection_dim)

        # Disciplinarity scorer
        self.disciplinarity_scorer = nn.Sequential(
            nn.Linear(projection_dim * 2, 256),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 1)
        )

    def encode_document(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor,
        field_ids: torch.Tensor,
        field_attention_mask: Optional[torch.Tensor] = None
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        # Encode document content
        doc_outputs = self.document_encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=True
        )
        doc_embedding = doc_outputs.last_hidden_state[:, 0, :]  # [CLS] token

        # Encode fields
        field_representations = self.field_encoder(field_ids, field_attention_mask)

        # Apply cross-attention between document and fields
        attended_fields, _ = self.cross_attention(
            query=doc_embedding.unsqueeze(1),
            key=field_representations,
            value=field_representations,
            key_padding_mask=None if field_attention_mask is None else ~field_attention_mask
        )

        # Project both embeddings
        doc_projection = self.content_projection(doc_embedding)
        field_projection = self.field_projection(attended_fields.squeeze(1))

        # Calculate disciplinarity score
        combined_features = torch.cat([doc_projection, field_projection], dim=-1)
        disciplinarity_score = self.disciplinarity_scorer(combined_features)

        return doc_projection, disciplinarity_score

    def encode_query(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor
    ) -> torch.Tensor:
        # For queries, we only need the content encoding
        query_outputs = self.document_encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=True
        )
        query_embedding = query_outputs.last_hidden_state[:, 0, :]
        query_projection = self.content_projection(query_embedding)
        return query_projection

    def score_document(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor,
        field_ids: torch.Tensor,
        field_attention_mask: Optional[torch.Tensor] = None
    ) -> Dict[str, torch.Tensor]:
        """
        Scores a single document for inference.
        """
        doc_embedding, disciplinarity_score = self.encode_document(
            input_ids,
            attention_mask,
            field_ids,
            field_attention_mask
        )

        return {
            'embedding': doc_embedding,
            'disciplinarity_score': disciplinarity_score
        }

    def forward(
        self,
        query_input_ids: torch.Tensor,
        query_attention_mask: torch.Tensor,
        doc_input_ids: torch.Tensor,
        doc_attention_mask: torch.Tensor,
        field_ids: torch.Tensor,
        field_attention_mask: Optional[torch.Tensor] = None,
        temperature: float = 0.07
    ) -> Dict[str, torch.Tensor]:
        # Encode query and document
        query_embedding = self.encode_query(query_input_ids, query_attention_mask)
        doc_embedding, disciplinarity_score = self.encode_document(
            doc_input_ids,
            doc_attention_mask,
            field_ids,
            field_attention_mask
        )

        # Calculate similarity scores
        similarity = F.cosine_similarity(
            query_embedding.unsqueeze(1),
            doc_embedding.unsqueeze(0),
            dim=-1
        ) / temperature

        loss_fn = create_combined_loss()
        loss = loss_fn(similarity, disciplinarity_score)

        return {
            'similarity': similarity,
            'disciplinarity_score': disciplinarity_score,
            'loss': loss
        }

In [ ]:
def create_combined_loss():
    """
    Creates the combined loss function for training.
    Combines contrastive loss with disciplinarity score.
    """
    def loss_fn(
        similarity: torch.Tensor,
        disciplinarity_score: torch.Tensor,
        alpha: float = 0.7
    ) -> torch.Tensor:
        # Contrastive loss (InfoNCE)
        exp_sim = torch.exp(similarity)
        log_prob = similarity - torch.log(exp_sim.sum(dim=-1, keepdim=True))
        contrastive_loss = -log_prob.diag().mean()

        # Disciplinarity regularization
        # We want to maximize the disciplinarity score
        disciplinarity_loss = -disciplinarity_score.mean()

        # Combine losses
        total_loss = alpha * contrastive_loss + (1 - alpha) * disciplinarity_loss
        return total_loss

    return loss_fn

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from typing import List, Dict, Tuple
import numpy as np
from collections import defaultdict

class FieldOfStudyProcessor:
    """
    Processes field of study data from SciRepEval and creates embeddings
    using a scientific language model.
    """
    def __init__(
        self,
        model_name: str = "allenai/scibert_scivocab_uncased",
        max_length: int = 128,
        device: str = "cuda" if torch.cuda.is_available() else "cpu"
    ):
        self.device = device
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name).to(device)
        self.max_length = max_length

        # Storage for processed data
        self.field_to_id: Dict[str, int] = {}
        self.id_to_field: Dict[int, str] = {}
        self.field_embeddings: torch.Tensor = None
        self.field_hierarchy: Dict[str, List[str]] = defaultdict(list)

    def build_field_vocabulary(self, dataset) -> None:
        """
        Creates a vocabulary of unique fields and their hierarchical relationships.
        """
        print("Building field vocabulary...")
        field_set = set()

        # Extract all unique fields and their relationships
        for example in dataset:
            fields = example['labels_text']
            for field in fields:
                field_set.add(field.lower())
                # If field contains 'and', it indicates a hierarchy
                if 'and' in field:
                    parent, child = field.lower().split('and', 1)
                    self.field_hierarchy[parent].append(child)

        # Create vocabulary mappings
        for idx, field in enumerate(sorted(field_set)):
            self.field_to_id[field] = idx
            self.id_to_field[idx] = field

        print(f"Found {len(self.field_to_id)} unique fields")
        print(f"Identified {len(self.field_hierarchy)} parent fields")

    def create_field_descriptions(self) -> Dict[str, str]:
        """
        Creates rich textual descriptions for each field incorporating hierarchical information.
        """
        descriptions = {}
        for field in self.field_to_id.keys():
            base_field = field.split('and')[0] if 'and' in field else field

            # Create description incorporating hierarchical information
            if 'and' in field:
                parent, child = field.split('and', 1)
                desc = f"Research field of {child} within the broader domain of {parent}"
                if self.field_hierarchy[parent]:
                    related = ', '.join(self.field_hierarchy[parent][:3])  # Top 3 related fields
                    desc += f". Related areas include {related}"
            else:
                desc = f"Research field of {field}"
                if self.field_hierarchy[field]:
                    subfields = ', '.join(self.field_hierarchy[field][:3])
                    desc += f" encompassing subfields such as {subfields}"

            descriptions[field] = desc

        return descriptions

    def compute_field_embeddings(self, batch_size: int = 32) -> None:
        """
        Computes embeddings for all fields using their descriptions.
        Incorporates hierarchical information into the embedding process.
        """
        print("Computing field embeddings...")
        descriptions = self.create_field_descriptions()

        # Prepare batches of fields
        fields = list(self.field_to_id.keys())
        num_batches = (len(fields) + batch_size - 1) // batch_size

        all_embeddings = []
        self.model.eval()

        with torch.no_grad():
            for i in range(num_batches):
                batch_fields = fields[i * batch_size:(i + 1) * batch_size]
                batch_descriptions = [descriptions[field] for field in batch_fields]

                # Tokenize descriptions
                encoded = self.tokenizer(
                    batch_descriptions,
                    padding=True,
                    truncation=True,
                    max_length=self.max_length,
                    return_tensors="pt"
                ).to(self.device)

                # Get embeddings
                outputs = self.model(**encoded)
                # Use [CLS] token embedding as field representation
                embeddings = outputs.last_hidden_state[:, 0, :]
                all_embeddings.append(embeddings.cpu())

        # Concatenate all embeddings
        self.field_embeddings = torch.cat(all_embeddings, dim=0)
        print(f"Created embeddings with shape: {self.field_embeddings.shape}")

    def get_field_embedding(self, field: str) -> torch.Tensor:
        """
        Retrieves the embedding for a specific field.
        """
        field = field.lower()
        if field not in self.field_to_id:
            raise ValueError(f"Unknown field: {field}")

        field_id = self.field_to_id[field]
        return self.field_embeddings[field_id]

    def get_related_fields(
        self,
        field: str,
        n: int = 5,
        similarity_threshold: float = 0.5
    ) -> List[Tuple[str, float]]:
        """
        Finds related fields based on embedding similarity.
        """
        field_embedding = self.get_field_embedding(field)

        # Compute similarities with all other fields
        similarities = torch.cosine_similarity(
            field_embedding.unsqueeze(0),
            self.field_embeddings,
            dim=1
        )

        # Get top-n similar fields
        top_n = torch.topk(similarities, k=min(n + 1, len(similarities)))

        related = []
        for idx, score in zip(top_n.indices[1:], top_n.values[1:]):  # Skip the first one (self)
            if score >= similarity_threshold:
                related.append((self.id_to_field[idx.item()], score.item()))

        return related

    def save_embeddings(self, path: str) -> None:
        """
        Saves the field embeddings and vocabulary mappings.
        """
        torch.save({
            'field_to_id': self.field_to_id,
            'id_to_field': self.id_to_field,
            'field_embeddings': self.field_embeddings,
            'field_hierarchy': dict(self.field_hierarchy)
        }, path)

    def load_embeddings(self, path: str) -> None:
        """
        Loads previously saved field embeddings and vocabulary mappings.
        """
        checkpoint = torch.load(path)
        self.field_to_id = checkpoint['field_to_id']
        self.id_to_field = checkpoint['id_to_field']
        self.field_embeddings = checkpoint['field_embeddings']
        self.field_hierarchy = defaultdict(list, checkpoint['field_hierarchy'])

In [ ]:
class FieldBatchCollator:
    """
    Collates fields of study into batches for the CrossDisciplinaryRanker.
    """
    def __init__(
        self,
        field_processor: FieldOfStudyProcessor,
        max_fields: int = 10
    ):
        self.field_processor = field_processor
        self.max_fields = max_fields

    def _pad_field_tensors(self, field_ids: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        current_length = field_ids.size(0)

        # If we have more fields than our maximum, truncate
        if current_length > self.max_fields:
            return field_ids[:self.max_fields], torch.ones(self.max_fields, dtype=torch.bool)

        # If we have fewer fields than maximum, pad with zeros
        elif current_length < self.max_fields:
            padding_length = self.max_fields - current_length
            padded_ids = torch.cat([
                field_ids,
                torch.zeros(padding_length, dtype=field_ids.dtype)
            ])
            # Create attention mask: 1s for real fields, 0s for padding
            attention_mask = torch.cat([
                torch.ones(current_length, dtype=torch.bool),
                torch.zeros(padding_length, dtype=torch.bool)
            ])
            return padded_ids, attention_mask

        # If we already have exactly max_fields
        else:
            return field_ids, torch.ones(self.max_fields, dtype=torch.bool)

    def __call__(self, batch: List[Dict]) -> Dict[str, torch.Tensor]:
        # Initialize lists to store batch elements
        query_input_ids = []
        query_attention_mask = []
        doc_input_ids = []
        doc_attention_mask = []
        field_ids_list = []
        field_attention_mask_list = []
        paper_ids = []

        # Process each example in the batch
        for example in batch:
            # Add query tensors
            query_input_ids.append(example['query_input_ids'])
            query_attention_mask.append(example['query_attention_mask'])

            # Add document tensors
            doc_input_ids.append(example['doc_input_ids'])
            doc_attention_mask.append(example['doc_attention_mask'])

            # Process and pad field information
            padded_fields, field_mask = self._pad_field_tensors(example['field_ids'])
            field_ids_list.append(padded_fields)
            field_attention_mask_list.append(field_mask)

            # Store paper IDs
            paper_ids.append(example['paper_id'])

        # Combine all tensors into batches using torch.stack
        return {
            'query_input_ids': torch.stack(query_input_ids),
            'query_attention_mask': torch.stack(query_attention_mask),
            'doc_input_ids': torch.stack(doc_input_ids),
            'doc_attention_mask': torch.stack(doc_attention_mask),
            'field_ids': torch.stack(field_ids_list),
            'field_attention_mask': torch.stack(field_attention_mask_list)
        }

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import get_linear_schedule_with_warmup
from typing import List, Dict, Optional, Tuple
import logging
import wandb
from tqdm import tqdm
import numpy as np
from sklearn.model_selection import train_test_split

class ScientificPaperDataset(Dataset):
    """
    Dataset class for scientific papers with their fields of study.
    Implements curriculum learning by controlling interdisciplinary complexity.
    """
    def __init__(
        self,
        papers: List[Dict],
        tokenizer,
        field_processor: FieldOfStudyProcessor,
        max_length: int = 512,
        curriculum_stage: int = 0
    ):
        self.papers = papers
        self.tokenizer = tokenizer
        self.field_processor = field_processor
        self.max_length = max_length
        self.curriculum_stage = curriculum_stage

        # Sort papers by interdisciplinary complexity
        self._sort_by_complexity()

    def _sort_by_complexity(self):
        """
        Sorts papers based on interdisciplinary complexity.
        Papers with more diverse fields are considered more complex.
        """
        def compute_complexity(paper):
            fields = paper['labels_text']
            # Count unique top-level fields
            top_fields = set(f.split('and')[0].lower() for f in fields)
            return len(top_fields)

        self.papers = sorted(self.papers, key=compute_complexity)

    def _get_curriculum_subset(self) -> List[Dict]:
        """
        Returns a subset of papers based on current curriculum stage.
        Higher stages include more complex interdisciplinary papers.
        """
        if self.curriculum_stage == 0:
            # Start with papers having few fields
            return self.papers[:len(self.papers)//3]
        elif self.curriculum_stage == 1:
            # Include papers with moderate interdisciplinary complexity
            return self.papers[:2*len(self.papers)//3]
        else:
            # Use all papers
            return self.papers

    def __len__(self) -> int:
        return len(self._get_curriculum_subset())

    def _create_query_from_title(self, title: str) -> str:
        """
        Creates a query-like representation from the title.
        This helps create query-document pairs for training.
        """
        return f"Find papers related to: {title}"

    def __getitem__(self, idx: int) -> Dict:
        paper = self._get_curriculum_subset()[idx]

        query_text = self._create_query_from_title(paper['title'])

        query_encoded = self.tokenizer(
            query_text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Tokenize paper title and abstract
        doc_text = f"{paper['title']} {paper['abstract']}"
        doc_encoded = self.tokenizer(
            doc_text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Process fields of study
        fields = paper['labels_text']
        field_ids = torch.tensor([
            self.field_processor.field_to_id[f.lower()]
            for f in fields
        ])

        return {
            'query_input_ids': query_encoded['input_ids'].squeeze(0),
            'query_attention_mask': query_encoded['attention_mask'].squeeze(0),
            'doc_input_ids': doc_encoded['input_ids'].squeeze(0),
            'doc_attention_mask': doc_encoded['attention_mask'].squeeze(0),
            'field_ids': field_ids,
            'field_attention_mask': torch.ones_like(field_ids, dtype=torch.bool),
            'paper_id': paper['doc_id']
        }

In [ ]:
class TrainingMonitor:
    """
    Monitors training progress and interdisciplinary learning.
    """
    def __init__(self, field_processor: FieldOfStudyProcessor, log_dir: str):
        self.field_processor = field_processor
        self.logger = self._setup_logger(log_dir)
        self.metrics = {
            'epoch_losses': [],
            'interdisciplinary_scores': [],
            'field_coverage': defaultdict(list)
        }

    def _setup_logger(self, log_dir: str) -> logging.Logger:
        logger = logging.getLogger('training_monitor')
        logger.setLevel(logging.INFO)
        handler = logging.FileHandler(f"{log_dir}/training.log")
        handler.setFormatter(logging.Formatter('%(asctime)s - %(message)s'))
        logger.addHandler(handler)
        return logger

    def log_batch(self, batch_metrics: Dict) -> None:
        """
        Logs metrics for a single batch.
        """
        self.logger.info(f"Batch metrics: {batch_metrics}")
        wandb.log(batch_metrics)

    def analyze_field_coverage(self, model, eval_loader) -> Dict:
        """
        Analyzes how well the model handles different field combinations.
        """
        field_pairs = defaultdict(list)
        model.eval()

        with torch.no_grad():
            for batch in eval_loader:
                scores = model(batch)['disciplinarity_score']
                for i, fields in enumerate(batch['fields']):
                    for f1 in fields:
                        for f2 in fields:
                            if f1 < f2:
                                field_pairs[f"{f1}-{f2}"].append(
                                    scores[i].item()
                                )

        return {k: np.mean(v) for k, v in field_pairs.items()}

In [ ]:
def save_model(model: nn.Module, output_dir, field_processor, tokenizer):
        """Saves the model and all necessary components."""
        print("Saving model artifacts...")

        # Save model state
        torch.save(model.state_dict(), f"{output_dir}/final_model.pt")

        # Save field processor
        field_processor.save_embeddings(
            f"{output_dir}/field_processor/embeddings.pt"
        )

        # Save tokenizer
        tokenizer.save_pretrained(f"{output_dir}/tokenizer")

        print(f"Model artifacts saved to {output_dir}")

def train_model(
    model: nn.Module,
    train_dataset: ScientificPaperDataset,
    val_dataset: ScientificPaperDataset,
    field_processor: FieldOfStudyProcessor,
    config: Dict,
    device: str = "cuda" if torch.cuda.is_available() else "cpu",
    output_dir: str = "./logs",
    tokenizer: AutoTokenizer = None
) -> nn.Module:
    """
    Trains the cross-disciplinary ranker with curriculum learning.
    """
    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate'])

    # Setup data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=config['batch_size'],
        shuffle=True,
        collate_fn=FieldBatchCollator(field_processor)
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=config['batch_size'],
        shuffle=False,
        collate_fn=FieldBatchCollator(field_processor)
    )

    # Initialize training monitor
    monitor = TrainingMonitor(field_processor, config['log_dir'])

    # Setup learning rate scheduler
    num_training_steps = len(train_loader) * config['num_epochs']
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=config['warmup_steps'],
        num_training_steps=num_training_steps
    )

    # Main training loop
    best_val_loss = float('inf')
    curriculum_patience = config['curriculum_patience']
    curriculum_counter = 0

    for epoch in range(config['num_epochs']):
        model.train()
        epoch_loss = 0

        with tqdm(train_loader, desc=f"Epoch {epoch+1}") as pbar:
            for batch in pbar:
                # Move batch to device
                batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v
                        for k, v in batch.items()}

                # Forward pass
                outputs = model(**batch)
                loss_function = create_combined_loss()
                loss = loss_function(outputs['similarity'], outputs['disciplinarity_score'])
                # loss = outputs['loss']

                # Backward pass
                optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), config['max_grad_norm'])
                optimizer.step()
                scheduler.step()

                # Update progress bar
                epoch_loss += loss.item()
                pbar.set_postfix({'loss': loss.item()})

                # Log metrics
                monitor.log_batch({
                    'loss': loss.item(),
                    'learning_rate': scheduler.get_last_lr()[0]
                })

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                batch = {k: v.to(device) if isinstance(v, torch.Tensor) else v
                        for k, v in batch.items()}
                outputs = model(**batch)
                loss_function = create_combined_loss()
                val_loss += loss_function(outputs['similarity'], outputs['disciplinarity_score'])

        val_loss /= len(val_loader)
        monitor.log_batch({'val_loss': val_loss})

        # Analyze field coverage
        # field_coverage = monitor.analyze_field_coverage(model, val_loader)
        # monitor.log_batch({'field_coverage': field_coverage})

        save_model(model, output_dir, field_processor, tokenizer) #remove

        # Curriculum learning update
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            curriculum_counter = 0
            torch.save(model.state_dict(), f"{config['log_dir']}/best_model.pt")
        else:
            curriculum_counter += 1

        if curriculum_counter >= curriculum_patience:
            curriculum_counter = 0
            train_dataset.curriculum_stage += 1
            monitor.logger.info(f"Advancing to curriculum stage {train_dataset.curriculum_stage}")

    return model

In [ ]:
default_config = {
    'batch_size': 32,
    'learning_rate': 2e-5,
    'num_epochs': 10,
    'warmup_steps': 1000,
    'max_grad_norm': 1.0,
    'curriculum_patience': 2,
    'log_dir': './logs'
}

In [ ]:
def prepare_test_data(dataset, num_samples=1000):
    """
    Prepares a small subset of the dataset for testing.
    Ensures we have a good mix of interdisciplinary papers.
    """
    # Convert to list for easier manipulation
    papers = list(dataset)

    # Sort papers by number of fields to ensure diverse sampling
    papers.sort(key=lambda x: len(x['labels_text']), reverse=True)

    # Take a stratified sample
    interdisciplinary = papers[:num_samples//2]  # Papers with more fields
    regular = papers[len(papers)//2:len(papers)//2 + num_samples//2]  # Papers with fewer fields

    return interdisciplinary + regular

In [ ]:
from datetime import datetime
import os
from datasets import load_dataset

def run_test():
    # Initialize wandb for experiment tracking
    run_id = datetime.now().strftime("%Y%m%d_%H%M%S")
    wandb.init(
        project="cross-disciplinary-ranker",
        name=f"test_run_{run_id}",
        config=default_config
    )

    print("Loading SciRepEval dataset...")
    dataset = load_dataset("allenai/scirepeval", "fos")['train']

    # Prepare test subset
    test_papers = prepare_test_data(dataset)
    print(f"Prepared {len(test_papers)} papers for testing")

    # Initialize tokenizer
    tokenizer = AutoTokenizer.from_pretrained("allenai/specter2_base")

    # Initialize field processor and compute embeddings
    print("Initializing field processor...")
    field_processor = FieldOfStudyProcessor()
    field_processor.build_field_vocabulary(test_papers)
    field_processor.compute_field_embeddings()

    # Create train/val split
    train_papers = test_papers[:800]
    val_papers = test_papers[800:]

    # Initialize datasets
    print("Creating datasets...")
    train_dataset = ScientificPaperDataset(
        train_papers,
        tokenizer,
        field_processor,
        curriculum_stage=0
    )
    val_dataset = ScientificPaperDataset(
        val_papers,
        tokenizer,
        field_processor,
        curriculum_stage=2  # No curriculum for validation
    )

    # Initialize model
    print("Initializing model...")
    model = CrossDisciplinaryRanker(
        base_model_name="allenai/specter2_base",
        field_embedding_dim=768,
        projection_dim=256
    )

    # Update field embeddings in model
    print("Updating model's field embeddings...")
    model.field_encoder.field_embeddings = nn.Embedding.from_pretrained(
        field_processor.field_embeddings,
        freeze=False
    )

    # Create log directory
    log_dir = f"./logs/test_run_{run_id}"
    os.makedirs(log_dir, exist_ok=True)

    # Update config for test run
    test_config = default_config.copy()
    test_config.update({
        'num_epochs': 3,  # Reduced epochs for testing
        'batch_size': 16,  # Smaller batch size
        'log_dir': log_dir
    })

    # Start training
    print("Starting training...")
    try:
        model = train_model(
            model=model,
            train_dataset=train_dataset,
            val_dataset=val_dataset,
            field_processor=field_processor,
            config=test_config
        )

        # Save final model and field processor
        torch.save(model.state_dict(), f"{log_dir}/final_model.pt")
        field_processor.save_embeddings(f"{log_dir}/field_embeddings.pt")

        # Test interdisciplinary ranking
        print("\nTesting interdisciplinary ranking...")
        model.eval()
        with torch.no_grad():
            # Select a test paper with multiple fields
            test_paper = next(p for p in val_papers if len(p['labels_text']) > 0)

            # Tokenize paper
            text = f"{test_paper['title']} {test_paper['abstract']}"
            encoded = tokenizer(
                text,
                padding='max_length',
                truncation=True,
                max_length=512,
                return_tensors='pt'
            )

            # Get predictions
            fields = test_paper['labels_text']
            field_ids = torch.tensor([
                [field_processor.field_to_id[f.lower()] for f in fields]
            ])
            field_mask = torch.ones_like(field_ids, dtype=torch.bool)

            outputs = model.score_document(
                encoded['input_ids'].cuda(),
                encoded['attention_mask'].cuda(),
                field_ids.cuda(),
                field_mask.cuda()
            )

            print("\nTest paper analysis:")
            print(f"Title: {test_paper['title']}")
            print(f"Fields: {', '.join(fields)}")
            print(f"Interdisciplinary score: {outputs['disciplinarity_score'].item():.3f}")

            # Get related fields
            print("\nRelated fields:")
            for field in fields:
                related = field_processor.get_related_fields(field.lower(), n=3)
                print(f"\n{field}:")
                for related_field, score in related:
                    print(f"  - {related_field}: {score:.3f}")

    except Exception as e:
        print(f"Error during training: {str(e)}")
        raise
    finally:
        wandb.finish()

In [ ]:
prod_config = {
    "base_model": "allenai/specter2_base",
    "field_embedding_dim": 768,
    "projection_dim": 256,
    "max_length": 512,
    "batch_size": 32,
    "learning_rate": 2e-5,
    "num_epochs": 10,
    "warmup_steps": 1000,
    "max_grad_norm": 1.0,
    "curriculum_patience": 2,
    'log_dir': './logs'
}

In [ ]:
import torch
from transformers import AutoTokenizer
from datasets import load_dataset
import wandb
from datetime import datetime
import os
import json
from sklearn.model_selection import train_test_split
from google.colab import drive

class ModelTrainer:
    """
    Handles the complete training pipeline for the cross-disciplinary ranker,
    including model checkpointing and evaluation.
    """
    def __init__(self, config_path: str = "training_config.json"):
        # Load configuration

        self.config = prod_config

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.setup_directories()
        self.initialize_wandb()

    def setup_directories(self):
        """Creates necessary directories for model artifacts."""
        self.run_id = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.output_dir = f"/content/drive/My Drive/Colab Notebooks/models/run_{self.run_id}"

        # Create directories for checkpoints and final model
        os.makedirs(f"{self.output_dir}/checkpoints", exist_ok=True)
        os.makedirs(f"{self.output_dir}/field_processor", exist_ok=True)

    def initialize_wandb(self):
        """Initializes Weights & Biases for experiment tracking."""
        wandb.init(
            project="cross-disciplinary-ranker",
            name=f"production_run_{self.run_id}",
            config=self.config
        )

    def prepare_data(self):
        """Loads and prepares the full dataset for training."""
        print("Loading and preparing dataset...")

        # Load full dataset
        dataset = load_dataset("allenai/scirepeval", "fos")['validation']
        all_papers = list(dataset)

        # Create train/validation/test split
        train_papers, test_papers = train_test_split(
            all_papers, test_size=0.1, random_state=42
        )
        train_papers, val_papers = train_test_split(
            train_papers, test_size=0.1, random_state=42
        )

        # Initialize tokenizer and field processor
        self.tokenizer = AutoTokenizer.from_pretrained(self.config['base_model'])
        self.field_processor = FieldOfStudyProcessor()
        self.field_processor.build_field_vocabulary(train_papers)
        self.field_processor.compute_field_embeddings()

        # Create datasets
        self.train_dataset = ScientificPaperDataset(
            train_papers,
            self.tokenizer,
            self.field_processor,
            max_length=self.config['max_length'],
            curriculum_stage=0
        )

        self.val_dataset = ScientificPaperDataset(
            val_papers,
            self.tokenizer,
            self.field_processor,
            max_length=self.config['max_length'],
            curriculum_stage=2  # No curriculum for validation
        )

        self.test_dataset = ScientificPaperDataset(
            test_papers,
            self.tokenizer,
            self.field_processor,
            max_length=self.config['max_length'],
            curriculum_stage=2
        )

        print(f"Dataset splits - Train: {len(train_papers)}, Val: {len(val_papers)}, Test: {len(test_papers)}")

    def train(self):
        """Executes the full training pipeline."""
        print("Initializing model...")
        self.model = CrossDisciplinaryRanker(
            base_model_name=self.config['base_model'],
            field_embedding_dim=self.config['field_embedding_dim'],
            projection_dim=self.config['projection_dim']
        ).to(self.device)

        # Update field embeddings
        self.model.field_encoder.field_embeddings = nn.Embedding.from_pretrained(
            self.field_processor.field_embeddings,
            freeze=False
        )

        print("Starting training...")
        trained_model = train_model(
            model=self.model,
            train_dataset=self.train_dataset,
            val_dataset=self.val_dataset,
            field_processor=self.field_processor,
            config=self.config,
            device=self.device,
            output_dir=self.output_dir, #remove
            tokenizer=self.tokenizer #remove
        )

        self.save_model(trained_model)
        self.evaluate_model(trained_model)

    def save_model(self, model: nn.Module):
        """Saves the model and all necessary components."""
        print("Saving model artifacts...")

        # Save model state
        torch.save(model.state_dict(), f"{self.output_dir}/final_model.pt")

        # Save field processor
        self.field_processor.save_embeddings(
            f"{self.output_dir}/field_processor/embeddings.pt"
        )

        # Save configuration
        with open(f"{self.output_dir}/config.json", 'w') as f:
            json.dump(self.config, f, indent=4)

        # Save tokenizer
        self.tokenizer.save_pretrained(f"{self.output_dir}/tokenizer")

        print(f"Model artifacts saved to {self.output_dir}")

    def evaluate_model(self, model: nn.Module):
        """Evaluates model on test set and logs metrics."""
        print("Evaluating model on test set...")
        model.eval()

        test_loader = DataLoader(
            self.test_dataset,
            batch_size=self.config['batch_size'],
            shuffle=False,
            collate_fn=FieldBatchCollator(self.field_processor)
        )

        metrics = defaultdict(list)

        with torch.no_grad():
            for batch_data in test_loader:
                model_inputs = batch_data
                model_inputs = {k: v.to(self.device) if isinstance(v, torch.Tensor) else v
                              for k, v in model_inputs.items()}

                outputs = model(**model_inputs)

                # Collect metrics
                metrics['loss'].append(outputs['loss'].item())
                metrics['disciplinarity_scores'].append(
                    outputs['disciplinarity_score'].cpu().numpy()
                )

        # Compute and log final metrics
        final_metrics = {
            'test_loss': np.mean(metrics['loss']),
            'avg_disciplinarity_score': np.mean(metrics['disciplinarity_scores']),
            'std_disciplinarity_score': np.std(metrics['disciplinarity_scores'])
        }

        wandb.log(final_metrics)
        print("Test set metrics:", final_metrics)

In [ ]:
trainer = ModelTrainer()
trainer.prepare_data()
trainer.train()

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer
from typing import List, Dict, Optional
import json

class CrossDisciplinaryReranker:
    """
    A component that reranks papers based on their cross-disciplinary relevance
    to a given query.
    """
    def __init__(self, model_dir: str):
        """
        Initializes the reranker with a trained model.

        Args:
            model_dir: Directory containing the saved model artifacts
        """
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.load_model(model_dir)

    def load_model(self, model_dir: str):
        """Loads all necessary components from the saved model directory."""
        # Load configuration
        self.config = prod_config

        # Initialize model
        self.model = CrossDisciplinaryRanker(
            base_model_name=self.config['base_model'],
            field_embedding_dim=self.config['field_embedding_dim'],
            projection_dim=self.config['projection_dim']
        ).to(self.device)

        self.field_processor = FieldOfStudyProcessor()
        self.field_processor.load_embeddings(
            f"{model_dir}/field_processor/embeddings.pt"
        )

        self.model.field_encoder.field_embeddings = nn.Embedding.from_pretrained(
        self.field_processor.field_embeddings.to(self.device),
        freeze=False
      )

        # Load model weights
        self.model.load_state_dict(
            torch.load(f"{model_dir}/final_model.pt", map_location=self.device)
        )
        self.model.eval()

        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(f"{model_dir}/tokenizer")

        # Load field processor
        self.field_processor = FieldOfStudyProcessor()
        self.field_processor.load_embeddings(
            f"{model_dir}/field_processor/embeddings.pt"
        )

    def rerank(
        self,
        query: str,
        papers: List[Dict],
        top_k: Optional[int] = None
    ) -> List[Dict]:
        """
        Reranks papers based on query relevance and cross-disciplinary importance.

        Args:
            query: The search query
            papers: List of paper dictionaries containing title, abstract, and fields
            top_k: Number of top papers to return (None for all)

        Returns:
            List of reranked papers with scores
        """
        # Encode query
        query_encoded = self.tokenizer(
            query,
            padding='max_length',
            truncation=True,
            max_length=self.config['max_length'],
            return_tensors='pt'
        ).to(self.device)

        paper_scores = []

        with torch.no_grad():
            for paper in papers:
                # Encode paper
                doc_text = f"{paper['title']} {paper['abstract']}"
                doc_encoded = self.tokenizer(
                    doc_text,
                    padding='max_length',
                    truncation=True,
                    max_length=self.config['max_length'],
                    return_tensors='pt'
                ).to(self.device)

                # Process fields
                field_ids = torch.tensor([
                    [self.field_processor.field_to_id[f.lower()]
                     for f in paper['fields'] if f is not None]
                ]).to(self.device)

                field_mask = torch.ones_like(field_ids, dtype=torch.bool)

                # Get model outputs
                outputs = self.model(
                    query_input_ids=query_encoded['input_ids'],
                    query_attention_mask=query_encoded['attention_mask'],
                    doc_input_ids=doc_encoded['input_ids'],
                    doc_attention_mask=doc_encoded['attention_mask'],
                    field_ids=field_ids,
                    field_attention_mask=field_mask
                )

                # Combine similarity and disciplinarity scores
                final_score = (outputs['similarity'].item() +
                             outputs['disciplinarity_score'].item()) / 2

                paper_scores.append((paper, final_score))

        # Sort papers by score
        paper_scores.sort(key=lambda x: x[1], reverse=True)

        # Select top-k if specified
        if top_k is not None:
            paper_scores = paper_scores[:top_k]

        # Return reranked papers with scores
        return [{
            **paper,
            'cross_disciplinary_score': score
        } for paper, score in paper_scores]

In [ ]:
papers = get_paper_title("quantum computing", 50)

In [ ]:
clean_papers = []

for paper in papers:
  if paper['fields'] is not None:
    clean_papers.append(paper)

In [ ]:
model_dir = f"/content/drive/My Drive/Colab Notebooks/models/run_20241211_123610"

reranker = CrossDisciplinaryReranker(model_dir)

reranked_results = reranker.rerank(
        query="quantum computing",
        papers=clean_papers,
        top_k=10  # Adjust as needed
    )

<ipython-input-4-7ee0e74a6b8c>:174: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path)
<ipython-input-14-93ee200936e7>:46: FutureWarning: You are us

In [ ]:
for results in reranked_results:
  result = {"Title": results['title'], "Fields": results['fields'], "Score": results['cross_disciplinary_score']}
  print(json.dumps(result, indent=4))

{
    "Title": "Quantum Chemistry in the Age of Quantum Computing.",
    "Fields": [
        "Medicine",
        "Computer Science",
        "Physics",
        "Chemistry"
    ],
    "Score": 128.73172903060913
}
{
    "Title": "Quantum computing enhanced computational catalysis",
    "Fields": [
        "Materials Science",
        "Physics",
        "Computer Science"
    ],
    "Score": 128.7284231185913
}
{
    "Title": "The state of quantum computing applications in health and medicine",
    "Fields": [
        "Physics",
        "Biology"
    ],
    "Score": 128.71499681472778
}
{
    "Title": "Quantum computing 40 years later",
    "Fields": [
        "Physics"
    ],
    "Score": 128.527765750885
}
{
    "Title": "Is quantum computing green? An estimate for an energy-efficiency quantum advantage",
    "Fields": [
        "Physics"
    ],
    "Score": 128.39431047439575
}
{
    "Title": "Emerging quantum computing algorithms for quantum chemistry",
    "Fields": [
        "Physi

In [ ]:
fos_data = load_dataset("allenai/scirepeval", "fos")['train']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

evaluation-00000-of-00001.parquet:   0%|          | 0.00/38.4M [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/153M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/152M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/38.4M [00:00<?, ?B/s]

Generating evaluation split:   0%|          | 0/68147 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/541218 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/67631 [00:00<?, ? examples/s]

In [ ]:
def evaluate_model(model: nn.Module, field_processor, device, tokenizer):
        """Evaluates model on test set and logs metrics."""
        print("Evaluating model on test set...")

        test_dataset = fos_data['evaluation']
        test_dataset = prepare_test_data(test_dataset, 3200)
        test_dataset = ScientificPaperDataset(
            test_dataset,
            tokenizer,
            field_processor,
            max_length=prod_config['max_length'],
            curriculum_stage=2
        )

        model.eval()

        test_loader = DataLoader(
            test_dataset,
            batch_size=prod_config['batch_size'],
            shuffle=False,
            collate_fn=FieldBatchCollator(field_processor)
        )

        metrics = defaultdict(list)

        with torch.no_grad():
            for batch_data in test_loader:
                model_inputs = batch_data
                model_inputs = {k: v.to(device) if isinstance(v, torch.Tensor) else v
                              for k, v in model_inputs.items()}

                outputs = model(**model_inputs)

                # Collect metrics
                metrics['loss'].append(outputs['loss'].item())
                metrics['disciplinarity_scores'].append(
                    outputs['disciplinarity_score'].cpu().numpy()
                )

        # Compute and log final metrics
        final_metrics = {
            'test_loss': np.mean(metrics['loss']),
            'avg_disciplinarity_score': np.mean(metrics['disciplinarity_scores']),
            'std_disciplinarity_score': np.std(metrics['disciplinarity_scores'])
        }

        #wandb.log(final_metrics)
        print("Test set metrics:", final_metrics)


model_dir = f"/content/drive/My Drive/Colab Notebooks/models/run_20241211_123610"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CrossDisciplinaryRanker(
            base_model_name=prod_config['base_model'],
            field_embedding_dim=prod_config['field_embedding_dim'],
            projection_dim=prod_config['projection_dim']
        ).to(device)

field_processor = FieldOfStudyProcessor()
field_processor.load_embeddings(
            f"{model_dir}/field_processor/embeddings.pt"
        )

model.field_encoder.field_embeddings = nn.Embedding.from_pretrained(
        field_processor.field_embeddings.to(device),
        freeze=False
    )

model.load_state_dict(
            torch.load(f"{model_dir}/final_model.pt", map_location=device)
        )

tokenizer = AutoTokenizer.from_pretrained(f"{model_dir}/tokenizer")

# evaluate_model(model, field_processor, device, tokenizer)

print("\nTesting interdisciplinary ranking...")
model.eval()
with torch.no_grad():
  # Select a test paper with multiple fields
  test_paper = fos_data[6]

  # Tokenize paper
  text = f"{test_paper['title']} {test_paper['abstract']}"
  encoded = tokenizer(
      text,
      padding='max_length',
      truncation=True,
      max_length=512,
      return_tensors='pt'
  )

  # Get predictions
  fields = test_paper['labels_text']
  field_ids = torch.tensor([
      [field_processor.field_to_id[f.lower()] for f in fields]
  ])
  field_mask = torch.ones_like(field_ids, dtype=torch.bool)

  outputs = model.score_document(
      encoded['input_ids'].cuda(),
      encoded['attention_mask'].cuda(),
      field_ids.cuda(),
      field_mask.cuda()
  )

  print("\nTest paper analysis:")
  print(f"Title: {test_paper['title']}")
  print(f"Fields: {', '.join(fields)}")
  print(f"Interdisciplinary score: {outputs['disciplinarity_score'].item():.3f}")

  # Get related fields
  print("\nRelated fields:")
  for field in fields:
      related = field_processor.get_related_fields(field.lower(), n=3)
      print(f"\n{field}:")
      for related_field, score in related:
          print(f"  - {related_field}: {score:.3f}")

<ipython-input-4-7ee0e74a6b8c>:174: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path)
<ipython-input-18-4369d4714733>:70: FutureWarning: You are us


Testing interdisciplinary ranking...

Test paper analysis:
Title: IRS Proposes Basis-Tracing Regulations
Fields: Business
Interdisciplinary score: 242.841

Related fields:

Business:
  - economics: 0.933
  - law: 0.928
  - education: 0.926


In [ ]:
if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not available.")

CUDA is available!
